# CBMC-Compatible Harness Generation System - Part 2: Core Tools

This part defines the primary tools for interacting with the system to analyze code, detect patterns, and verify harnesses.

In [1]:
# Real code embedding tool using ChromaDB
@tool
def embed_code(code: str) -> Dict[str, Any]:
    """Embeds source code and stores it in the ChromaDB embedding database.
    
    Args:
        code: The source code to embed
    
    Returns:
        A dictionary of embedded code representations
    """
    # Parse the code to extract functions
    functions = {}
    
    # Simple parser to extract function names and bodies
    import re
    func_pattern = r"(\w+)\s+([\w_]+)\s*\(([^)]*)\)\s*\{([^}]*)\}"
    matches = re.findall(func_pattern, code)
    
    # Clear existing collection
    code_collection.delete(where={})
    
    # Process each function
    function_ids = []
    function_texts = []
    function_metadatas = []
    
    for match in matches:
        return_type, func_name, params, body = match
        full_text = f"{return_type} {func_name}({params}) {{{body}}}"
        
        # Store function details in our return dictionary
        functions[func_name] = {
            "return_type": return_type,
            "params": params.strip(),
            "body": body.strip(),
            "full_text": full_text
        }
        
        # Check for malloc/free in the body to add to metadata
        has_malloc = "malloc(" in body
        has_free = "free(" in body
        
        # Prepare for ChromaDB
        function_ids.append(func_name)
        function_texts.append(full_text)
        function_metadatas.append({
            "name": func_name,
            "return_type": return_type,
            "params": params.strip(),
            "has_malloc": has_malloc,
            "has_free": has_free,
            "allocation_without_free": has_malloc and not has_free
        })
    
    # Add to ChromaDB
    if function_ids:
        code_collection.add(
            ids=function_ids,
            documents=function_texts,
            metadatas=function_metadatas
        )
    
    return {
        "functions": functions,
        "message": f"Successfully embedded {len(functions)} functions in ChromaDB"
    }

In [2]:
# Real pattern database tool using ChromaDB
@tool
def query_pattern_db(query: str) -> Dict[str, Any]:
    """Queries the ChromaDB pattern database for known memory leak patterns.
    
    Args:
        query: The query string or function to match against patterns
    
    Returns:
        A dictionary of matching patterns
    """
    # Query the pattern collection to find relevant patterns
    results = pattern_collection.query(
        query_texts=[query],
        n_results=3  # Get the top 3 matching patterns
    )
    
    # Also check via metadata if the function contains known indicators
    if "malloc(" in query and "free(" not in query:
        # Direct metadata match for malloc without free
        direct_match = "malloc_without_free"
    elif "if" in query and "free(" in query:
        # Potential conditional free
        direct_match = "conditional_free"
    elif query.count("malloc(") > 1:
        # Multiple malloc calls
        direct_match = "nested_malloc"
    else:
        direct_match = None
    
    # Process results
    matching_patterns = {}
    
    # Add patterns from semantic search
    for i, (pattern_id, metadata, distance) in enumerate(zip(
        results['ids'][0],
        results['metadatas'][0],
        results['distances'][0]
    )):
        # Only include if reasonably close in embedding space
        if distance < 0.3:
            matching_patterns[metadata['name']] = {
                "description": metadata["description"],
                "severity": metadata["severity"],
                "verification_strategy": metadata["verification_strategy"],
                "similarity_score": 1.0 - distance  # Convert to similarity
            }
    
    # Add direct match if found and not already included
    if direct_match and direct_match not in matching_patterns:
        # Find the metadata for this pattern
        for metadata in pattern_collection.get(ids=[f"pattern{['malloc_without_free', 'nested_malloc', 'conditional_free'].index(direct_match) + 1}"])['metadatas']:
            if metadata['name'] == direct_match:
                matching_patterns[direct_match] = {
                    "description": metadata["description"],
                    "severity": metadata["severity"],
                    "verification_strategy": metadata["verification_strategy"],
                    "similarity_score": 1.0  # Direct match gets perfect score
                }
                break
    
    return {
        "matching_patterns": matching_patterns,
        "message": f"Found {len(matching_patterns)} potential matching patterns via ChromaDB"
    }

In [3]:
# CBMC verification tool (accessing source code directly)
@tool
def run_cbmc(function_name: str, harness_code: str) -> Dict[str, Any]:
    """Runs CBMC verification on the provided harness code.
    
    Args:
        function_name: The name of the function being verified
        harness_code: The harness code to be verified by CBMC
    
    Returns:
        Results of the CBMC verification
    """
    # In a real implementation, this would call the actual CBMC tool
    # Get the function code from ChromaDB
    function_result = code_collection.get(
        ids=[function_name],
        include=["documents", "metadatas"]
    )
    
    if not function_result["ids"]:
        return {
            "function": function_name,
            "status": "ERROR",
            "message": f"Function '{function_name}' not found in code database.",
            "suggestions": "Verify the function name is correct."
        }
    
    # Get the function code
    function_code = function_result["documents"][0]
    function_metadata = function_result["metadatas"][0]
    
    # Analyze potential issues based on the function and harness
    has_malloc = function_metadata.get("has_malloc", False)
    has_free = function_metadata.get("has_free", False)
    
    # Prepare combined code for simulation
    # In a real implementation, this would be written to a file and passed to CBMC
    combined_code = f"""
    {function_code}
    
    /* CBMC Harness */
    {harness_code}
    """
    
    # Simulate CBMC verification based on code analysis
    if has_malloc and not has_free:
        # Function allocates memory but doesn't free it
        if "process_data" in function_name and "size > 100" in function_code:
            return {
                "function": function_name,
                "status": "FAILED",
                "message": "VERIFICATION FAILED: Memory leak detected in conditional path when size > 100.",
                "suggestions": "Ensure memory is freed in all execution paths."
            }
        elif "allocate_buffer" in function_name or "duplicate_string" in function_name:
            # These functions are allocation functions by design
            if "__CPROVER_assume" in harness_code and "free(" in harness_code:
                return {
                    "function": function_name,
                    "status": "SUCCESS",
                    "message": "VERIFICATION SUCCESSFUL: Allocation function verified with proper deallocation in harness.",
                    "suggestions": "Function is designed to allocate memory; ensure callers free the memory."
                }
            else:
                return {
                    "function": function_name,
                    "status": "WARNING",
                    "message": "VERIFICATION WARNING: Allocation function - harness should verify proper allocation and check for NULL.",
                    "suggestions": "Add appropriate assertions to verify allocation behavior."
                }
        else:
            return {
                "function": function_name,
                "status": "FAILED",
                "message": "VERIFICATION FAILED: Memory leak detected. Allocated memory not freed.",
                "suggestions": "Ensure all allocated memory is freed before function exit."
            }
    elif "NULL" not in harness_code and has_malloc:
        return {
            "function": function_name,
            "status": "WARNING",
            "message": "VERIFICATION WARNING: Harness doesn't check for NULL pointer returns.",
            "suggestions": "Add NULL pointer checks in the harness."
        }
    else:
        return {
            "function": function_name,
            "status": "SUCCESS",
            "message": "VERIFICATION SUCCESSFUL: No memory leaks detected.",
            "suggestions": ""
        }

In [4]:
# Create a list of tools and bind them to the LLM
tools = [embed_code, query_pattern_db, run_cbmc]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)